In [40]:
import numpy as np
import os,glob
import scipy.ndimage as sni
import sys
import re
import itertools
from shutil import copytree
from astropy.wcs import WCS
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from astropy.visualization import make_lupton_rgb
import aplpy
from astropy.nddata import Cutout2D
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.visualization.wcsaxes import SphericalCircle
from matplotlib.patches import Arrow
from photutils import SkyCircularAnnulus
from photutils import SkyCircularAperture
from photutils import CircularAperture
from photutils import CircularAnnulus
from photutils import SkyEllipticalAperture
from photutils import SkyEllipticalAnnulus
from photutils import EllipticalAperture
from photutils import EllipticalAnnulus
from photutils import aperture_photometry
import numpy.ma as ma
import math
import shutil
import pandas as pd

In [41]:
Dir='/home/heh15/workingspace/Arp240/NGC5258/ratio/'
scriptDir=Dir+'script/measure_1213/'
workDir=Dir+'mask/'
imageDir=Dir+'image/'
logDir=Dir+'log/'
ratioDir=Dir+'1213/contsub/'
maskDir=Dir+'mask/'

In [42]:
# function
def fits_import(fitsimage):
    hdr = fits.open(fitsimage)[0].header
    wcs = WCS(hdr).celestial
    data=fits.open(fitsimage)[0].data
    data=np.squeeze(data)
    data_masked=np.ma.masked_invalid(data)

    return wcs, data_masked

def masked_convert(data_masked,region_masked):
    data_mask=data_masked.mask
    region_mask=np.ma.make_mask(region_masked==0)
    region_mask=np.ma.mask_or(data_mask,region_mask)
    data_region=np.ma.masked_where(region_mask,data_masked)
    return data_region

def flux_mask_get(data_region,rms,chans,chan_width):
    flux=np.ma.sum(data_region)/beam_area_pix
    chans_tmp=chans+np.zeros((np.shape(data_region)[0],np.shape(data_region)[1]))
    error=np.sqrt(chans_tmp)*rms*chan_width/math.sqrt(beam_area_pix)
    error_masked=np.ma.masked_where(data_region.mask,error)
    uncertainty=math.sqrt(np.ma.sum(np.power(error_masked,2)))
    return flux, uncertainty

def ratio_cal(flux1,flux2,uncertainty1,uncertainty2):
    ratio=flux1/flux2
    uncertainty=math.sqrt((uncertainty1/flux1)**2+(uncertainty2/flux2)**2+2*0.05**2)*ratio
    return ratio, uncertainty

In [43]:
beam_area=2.186*1.896*1.1331
beam_area_pix=beam_area/0.09
regions=['south','rest','all']
values=['12CO10 flux','12CO10 uncertainty','13CO10 flux', '13CO10 uncertainty', '12CO21 flux',
        '12CO21 uncertainty','12/13','12/13 uncertainty','21/10','21/10 uncertainty']
CO13=['13CO10 flux', '13CO10 uncertainty']
ratio_table=pd.DataFrame(index=regions,columns=values)

values2=['12CO10 intensity','12CO10 uncertainty','13CO10 intensity','13CO10 uncertainty',
         '12CO21 intensity','12CO21 uncertainty']
intensities=pd.DataFrame(columns=values2,index=regions)

## 12CO 1-0

In [44]:
fitsimage=imageDir+'12CO10/NGC5258_12CO10_combine_contsub_pbcor_smooth_mom0.fits'
wcs=fits_import(fitsimage)[0]
data_masked=fits_import(fitsimage)[1]

In [45]:
# south
maskimage=maskDir+'south_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0016
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO10 flux']['south']=flux
ratio_table['12CO10 uncertainty']['south']=uncertainty

# rest
maskimage=maskDir+'rest_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0016
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO10 flux']['rest']=flux
ratio_table['12CO10 uncertainty']['rest']=uncertainty

# whole
maskimage=maskDir+'whole_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0016
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO10 flux']['all']=flux
ratio_table['12CO10 uncertainty']['all']=uncertainty

In [46]:
# south
maskimage=maskDir+'south_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO10 intensity']['south']=intensity
intensities['12CO10 uncertainty']['south']=intensity*ratio_table['12CO10 uncertainty']['south']\
                                  /ratio_table['12CO10 flux']['south']

# rest
maskimage=maskDir+'rest_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO10 intensity']['rest']=intensity
intensities['12CO10 uncertainty']['rest']=intensity*ratio_table['12CO10 uncertainty']['rest']\
                                  /ratio_table['12CO10 flux']['rest']

# whole
maskimage=maskDir+'whole_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO10 intensity']['all']=intensity
intensities['12CO10 uncertainty']['all']=intensity*ratio_table['12CO10 uncertainty']['rest']\
                                  /ratio_table['12CO10 flux']['rest']

In [47]:
print(intensities)

      12CO10 intensity 12CO10 uncertainty 13CO10 intensity 13CO10 uncertainty  \
south           5.2494          0.0335045              NaN                NaN   
rest           1.91646          0.0121293              NaN                NaN   
all            2.30265          0.0145735              NaN                NaN   

      12CO21 intensity 12CO21 uncertainty  
south              NaN                NaN  
rest               NaN                NaN  
all                NaN                NaN  


## 13CO 1-0 

In [48]:
fitsimage=imageDir+'13CO10/NGC5258_13CO10_contsub_pbcor_smooth_mom0.fits'
wcs=fits_import(fitsimage)[0]
data_masked=fits_import(fitsimage)[1]

Import the count channel map for 13CO1-0

In [49]:
fitsimage=imageDir+'13CO10/NGC5258_13CO10_12m_smooth_nchan.fits'
wcs=fits_import(fitsimage)[0]
chans_masked=fits_import(fitsimage)[1]
chans=chans_masked.data

In [50]:
# south
maskimage=maskDir+'south_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00062
chan_width=20
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['13CO10 flux']['south']=flux
ratio_table['13CO10 uncertainty']['south']=uncertainty

# rest
maskimage=maskDir+'rest_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00062
chan_width=20
chans=25
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['13CO10 flux']['rest']=flux
ratio_table['13CO10 uncertainty']['rest']=uncertainty

# whole
maskimage=maskDir+'whole_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00062
chan_width=20
chans=25
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['13CO10 flux']['all']=flux
ratio_table['13CO10 uncertainty']['all']=uncertainty

Calculate the intensity of the 13CO 1-0

In [51]:
# south
maskimage=maskDir+'south_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00062
chan_width=20
intensity=np.ma.mean(region)
intensities['13CO10 intensity']['south']=intensity
intensities['13CO10 uncertainty']['south']=intensity*ratio_table['13CO10 uncertainty']['south']\
                                  /ratio_table['13CO10 flux']['south']

# rest
maskimage=maskDir+'rest_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00062
chan_width=20
chans=25
intensity=np.ma.mean(region)
intensities['13CO10 intensity']['rest']=intensity
intensities['13CO10 uncertainty']['rest']=intensity*ratio_table['13CO10 uncertainty']['rest']\
                                  /ratio_table['13CO10 flux']['rest']

# whole
maskimage=maskDir+'whole_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00062
chan_width=20
chans=25
intensity=np.ma.mean(region)
intensities['13CO10 intensity']['all']=intensity
intensities['13CO10 uncertainty']['all']=intensity*ratio_table['13CO10 uncertainty']['rest']\
                                  /ratio_table['13CO10 flux']['rest']

In [52]:
print(intensities)

      12CO10 intensity 12CO10 uncertainty 13CO10 intensity 13CO10 uncertainty  \
south           5.2494          0.0335045         0.282825         0.00788714   
rest           1.91646          0.0121293         0.138339         0.00781533   
all            2.30265          0.0145735         0.159162         0.00899175   

      12CO21 intensity 12CO21 uncertainty  
south              NaN                NaN  
rest               NaN                NaN  
all                NaN                NaN  


## 12CO21 

In [53]:
fitsimage=imageDir+'12CO21/NGC5258_12CO21_combine_uvtaper_pbcor_smooth_mom0.fits'
wcs=fits_import(fitsimage)[0]
data_masked=fits_import(fitsimage)[1]

In [54]:
# south
maskimage=maskDir+'south_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0031
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO21 flux']['south']=flux
ratio_table['12CO21 uncertainty']['south']=uncertainty

# rest
maskimage=maskDir+'rest_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0031
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO21 flux']['rest']=flux
ratio_table['12CO21 uncertainty']['rest']=uncertainty

# whole
maskimage=maskDir+'whole_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0031
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO21 flux']['all']=flux
ratio_table['12CO21 uncertainty']['all']=uncertainty

In [55]:
# south
maskimage=maskDir+'south_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO21 intensity']['south']=intensity
intensities['12CO21 uncertainty']['south']=intensity*ratio_table['12CO21 uncertainty']['south']\
                                  /ratio_table['12CO21 flux']['south']

# rest
maskimage=maskDir+'rest_SFR_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO21 intensity']['rest']=intensity
intensities['12CO21 uncertainty']['rest']=intensity*ratio_table['12CO21 uncertainty']['rest']\
                                  /ratio_table['12CO21 flux']['rest']

# whole
maskimage=maskDir+'whole_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO21 intensity']['all']=intensity
intensities['12CO21 uncertainty']['all']=intensity*ratio_table['12CO21 uncertainty']['rest']\
                                  /ratio_table['12CO21 flux']['rest']

In [56]:
ratio_table['12/13']=ratio_table['12CO10 flux']/ratio_table['13CO10 flux']
fraction=((ratio_table['12CO10 uncertainty']/ratio_table['12CO10 flux'])**2+
                                  (ratio_table['13CO10 uncertainty']/ratio_table['13CO10 flux'])**2)**0.5
ratio_table['12/13 uncertainty']=ratio_table['12/13']*fraction

In [57]:
ratio_table['21/10']=ratio_table['12CO21 flux']/ratio_table['12CO10 flux']*115.27**2/230.54**2
fraction=((ratio_table['12CO21 uncertainty']/ratio_table['12CO21 flux'])**2+
                                  (ratio_table['12CO10 uncertainty']/ratio_table['12CO10 flux'])**2)**0.5
ratio_table['21/10 uncertainty']=ratio_table['21/10']*fraction
print(ratio_table)

      12CO10 flux 12CO10 uncertainty 13CO10 flux 13CO10 uncertainty  \
south     59.8566           0.382038     2.99729          0.0835854   
rest       166.74             1.0553     8.70628           0.491854   
all       226.597            1.12232     11.7036           0.531656   

      12CO21 flux 12CO21 uncertainty    12/13 12/13 uncertainty     21/10  \
south     195.371           0.740198  19.9702           0.57131  0.815995   
rest      486.482            2.04419  19.1517           1.08873  0.729401   
all       681.853            2.17408  19.3614          0.884737  0.752275   

      21/10 uncertainty  
south        0.00605659  
rest         0.00554119  
all          0.00443129  


In [58]:
print(intensities)

      12CO10 intensity 12CO10 uncertainty 13CO10 intensity 13CO10 uncertainty  \
south           5.2494          0.0335045         0.282825         0.00788714   
rest           1.91646          0.0121293         0.138339         0.00781533   
all            2.30265          0.0145735         0.159162         0.00899175   

      12CO21 intensity 12CO21 uncertainty  
south          17.1339           0.064915  
rest           5.59392          0.0235056  
all             6.9316          0.0291265  
